![Spider Ninja](static/images/NynLOGO.png)


# CKK Product Spider / Scripts 

In [14]:
import requests
from lxml import html
import os
import glob
import urllib
import arrow
from requests_html import HTMLSession
import datetime

* fetch sitemap for each site
* determine what urls to crawl
* Load crawl data into django
* Tags
* Split-View Template
* DIY Configurator template


In [15]:
#  Initiate Session!
session = HTMLSession()

    

In [16]:
def fetch_map(url):
    '''Fetch sitemap and return parsed txt'''
    response=session.get(url)
    return response

def parse_xml(response,days=-7):
    '''Parse response and return new URLS to crawl.
        Returns list of urls added in the last 7 days
    '''
    urls = response.html.xpath('//loc/text()')
    dates = [arrow.get(date) for date in response.html.xpath('//lastmod/text()')]
    urls_to_crawl=[]
    for x in enumerate(dates):
        if x[1].date() - datetime.timedelta(days) > datetime.date.today():
            urls_to_crawl.append(urls[x[0]])
    return urls_to_crawl



###### "kk", "hs", "both"

In [17]:
 
def get_urls_to_crawl():
    '''
    Fetch sitemaps and determine what urls to crawl.
    Return a dict of product urls per shop / sitemap
    '''
    kk_sitemap = 'https://knifekits.com/vcom/smproducts.xml'
    hs_sitemap = 'https://holstersmith.com/vcom/smproducts.xml'

    # parse_xml defaults to the last 7 days.
    # This can be adjusted (days=-7), Negative values are in the past!
    kk_urls =  parse_xml(fetch_map(kk_sitemap))
    hs_urls =  parse_xml(fetch_map(hs_sitemap)) 
    
    URLS = kk_urls.extend(hs_urls)
    print(URLS)
    
    results = {"kk":kk_urls,"hs":hs_urls,"both":[kk_urls,hs_urls]}
    return results


urls=get_urls_to_crawl()

    

None


In [18]:
print(urls)

{'kk': ['https://www.knifekits.com/vcom/product_info.php?products_id=9793', 'https://www.knifekits.com/vcom/product_info.php?products_id=1517', 'https://www.knifekits.com/vcom/product_info.php?products_id=2817', 'https://www.knifekits.com/vcom/product_info.php?products_id=957', 'https://www.knifekits.com/vcom/product_info.php?products_id=1760', 'https://www.knifekits.com/vcom/product_info.php?products_id=295', 'https://www.knifekits.com/vcom/product_info.php?products_id=605', 'https://www.knifekits.com/vcom/product_info.php?products_id=296', 'https://www.knifekits.com/vcom/product_info.php?products_id=8210', 'https://www.knifekits.com/vcom/product_info.php?products_id=7013', 'https://www.knifekits.com/vcom/product_info.php?products_id=8211', 'https://www.knifekits.com/vcom/product_info.php?products_id=8209', 'https://www.knifekits.com/vcom/product_info.php?products_id=7014', 'https://www.knifekits.com/vcom/product_info.php?products_id=16106', 'https://www.knifekits.com/vcom/product_inf

In [49]:
def parse_url(url):
    '''Parse url for product data
        Return dict(data)
    '''
    page = session.get(url)
    name = page.html.xpath('//h1/descendant-or-self::span[@itemprop="name"]/text()')
    raw_desc=page.html.xpath('//div[@itemprop="description"]')[0]
    description = page.html.xpath('//div[@itemprop="description"]/descendant-or-self::text()')
    price=page.html.xpath('//*[@itemprop="price"]/@content')[0]
    sku=page.html.xpath(".//*[@itemprop='model']/text()")[0]
    
    keywords = page.html.xpath(".//meta[@name='keywords']/@content")
    breadcrumbs = page.html.xpath('//*[@class="breadcrumb"]/descendant::text()')[2::2]
    image = page.html.xpath('.//div[@class="piGalMain"]/img/@src')[0]
    all_images = page.html.xpath('//img/@src')
    images = [img for img in all_images if "600" in img or "800" or "550" in img and ".jpg" in img]
    cat = breadcrumbs[1]
    subcat=breadcrumbs[2]
    Data = {"raw_desc": raw_desc,
            

            "name": name,
            "description": description,
            "price":price,
            "sku":sku,
            "keywords":keywords,
            "breadcrumbs":breadcrumbs,
            "image":image,
            "images":images,
            "link":url,
            "category":cat,
            "subcategory":subcat
            }
    return Data


### Get new product urls for each shop / sitemap

In [50]:
data = parse_url(urls['kk'][10])

### Fetch Product data from a url

#### Django Model: 
* raw_desc (summernote field)
* description (textfield)
* name
* image urlfield
* images list of urlfields
* description: description txt by line
* price 
* url
* breadcrumbs
* date_crawled Datetime
* modified_date date modified
* shop foreignkey!?



In [21]:
# "raw_desc": 
# "description": [str,]
# "price":price,
# "sku":sku,
# "keywords":keywords,
# "breadcrumbs":breadcrumbs,
# "image":image,
# "images":images,
# "link":url,
# "category":cat,
# "subcategory":subcat

data.keys()


dict_keys(['raw_desc', 'name', 'description', 'price', 'sku', 'keywords', 'breadcrumbs', 'image', 'images', 'link', 'category', 'subcategory'])

## Search for a SKU

In [47]:

def load_crawled_item_into_django(data,*args,**kwargs):
    item={}
    for key in data.keys():
        item[key]=data[key]
    
    shop="Knifekits.com"
    Product.objects.get_or_create(
         sku=item['sku'],
         description=item['raw_desc'],
         desc_text=item['description'],
         price=item['price'],
         url=item['link'],
         metaKeywords=item['keywords'],
         image=item['image'],
         images=item['image'],
         breadcrumbs=item['breadcrumbs'],
         name=item['name'],
         category=item['category'],
         subcat=item['subcat'],
         )
    return item




In [51]:
Product.objects.create(name=data['name'],sku=data['sku'],image=data['image'],images=data['images'],description=data['raw_desc'],desc_txt=data['description'],price=data['price'],breadcrumbs=data['breadcrumbs'],url=data['link'],category=data['category'],subcat=data['subcat'])

NameError: name 'Product' is not defined

In [52]:
def check_for_sku(shop_url, sku):
    '''
    Send a POST req to shop_url's search box
    If SKU is found, fetch the product_detail page and return it
    '''
    session.post(shop_url + sku)
    ###
    
#         MATCH = search_results.html.find('a[@itemprop="url"]', first=True)
#         if not MATCH:
#             print("No Matches!")
#             return False
#         else:
#             print("Match Found!", MATCH.attrs['href'])
#             return ses.get(MATCH.attrs['href'])

In [16]:
print(r.status_code)


200


In [22]:
import requests
sku='3975'
r=requests.post('https://knifekits.com/vcom/advanced_search_result.php?keywords={}'.format('KK706BL-BLDWD-K'))

results_page=html.fromstring(r.content)
if results_page.xpath('//*[@class="alert alert-info"]'):
    print("No match")
else:
    m=results_page.xpath("//a[@itemprop='url']/descendant-or-self::a/@href")
    print(m[0])
m=results_page.xpath('//*[@itemprop="url"]')[0]
m.attrib['href']     

https://www.knifekits.com/vcom/gentlemans-companion-combo-knife-kit-diy-blade-blank-wbloodwood-p-14556.html?osCsid=764blpj9rhlsosqe27a4db8984


'https://www.knifekits.com/vcom/gentlemans-companion-combo-knife-kit-diy-blade-blank-wbloodwood-p-14556.html?osCsid=764blpj9rhlsosqe27a4db8984'